## Notebook Purpose

Recreate deep decoder experiments run in `ConvDecoder_vs_DIP_vs_DD_multicoil.ipynb`, hereon referred to as the original notebook, which was extremely messy and unnecessarily complicated.

In [1]:
import os, sys
import h5py
import numpy as np
import torch
import matplotlib.pyplot as plt

# import warnings
# warnings.filterwarnings('ignore')

from utils.transform import to_tensor, to_np, np_to_var, apply_mask, ifft_2d
from utils.helpers import num_params
from include.decoder_conv import convdecoder
from include.mri_helpers import get_scale_factor
from include.helpers import np_to_var

# TODO: fix these imports
# from include import * 
# from include import transforms as transform
# from common.evaluate import * 
# from torch.autograd import Variable

In [2]:
if torch.cuda.is_available():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    dtype = torch.cuda.FloatTensor
    torch.cuda.set_device(0)
#     print("num GPUs",torch.cuda.device_count())
else:
    dtype = torch.FloatTensor

### Load MRI measurements, y

Isolate individual 2D slice

In [3]:
filename = '/bmrNAS/people/dvv/multicoil_test_v2/file1000781_v2.h5'
f = h5py.File(filename, 'r') 
# print('h5 file keys: ', f.keys())
print('k-space shape (num_slices, num_coils, x, y): ', f['kspace'].shape)

# isolate central k-space slice
slice_idx = f['kspace'].shape[0] // 2
ksp_slice = f['kspace'][slice_idx]
# note: didn't add tensor version e.g. slice_ksp_torchtensor in original

k-space shape (num_slices, num_coils, x, y):  (37, 15, 640, 368)


### Load mask, M

- Format of loaded mask is 1d binary vector of size ~368, i.e. sampling of vertical lines in image
- Convert mask to 0's and 1's, zero pad, convert to 2D, create torch transform

Note: See original notebook for generating a new mask, e.g. if .h5 doesn't have a mask

In [4]:
try:
    mask1d = np.array([1 if e else 0 for e in f["mask"]]) # load 1D binary mask
except:
    print('Implement method for generating a mask')
    sys.exit()
    
# zero out mask in outer regions e.g. mask and data have last dimn 368, but actual data is size 320
# TODO: if actual data is size 320, then why do we have dimn 368?
idxs_zero = (mask1d.shape[-1] - 320) // 2 # e.g. zero first/last (368-320)/2=24 indices
mask1d[:idxs_zero], mask1d[-idxs_zero:] = 0, 0

# create 2d mask. zero pad if dimensions don't line up - is this necessary?
mask2d = np.repeat(mask1d[None,:], ksp_slice.shape[1], axis=0)#.astype(int)
mask2d = np.pad(mask2d, ((0,),((ksp_slice.shape[-1]-mask2d.shape[-1])//2,)), mode='constant')

# convert shape e.g. (368,) --> (1, 1, 368, 1)
mask = to_tensor(np.array([[mask2d[0][np.newaxis].T]])).type(torch.FloatTensor)
print('under-sampling factor:', round(len(mask1d) / sum(mask1d), 2))

under-sampling factor: 8.98


### Set up ConvDecoder

In [5]:
arch_name = 'ConvDecoder'

in_size = [8,4]
out_size = ksp_slice.shape[1:] # shape of (x,y) image slice, e.g. (640, 368)
out_depth = ksp_slice.shape[0]*2 # 2*n_c, i.e. 2*15=30 if multi-coil
num_layers = 8
strides = [1]*(num_layers-1)
num_channels = 160
kernel_size = 3

net = convdecoder(in_size, out_size, out_depth, num_layers, strides, num_channels).type(dtype)

print('# parameters of {}:'.format(arch_name),num_params(net))
#print(net)

[(15, 8), (28, 15), (53, 28), (98, 53), (183, 102), (343, 193), (640, 368)]
# parameters of ConvDecoder: 1850560


### Fit ConvDecoder

# TODO: write this sequentially, e.g. outside of function

In [6]:
# fix the scaling b/w original image and random output image = net(input tensor w values ~U[0,1]) 
# e.g. scaling_factor = 168813
# note: can be done using the under-sampled kspace, but we use the full kspace
scaling_factor, net_input = get_scale_factor(net,
                                   num_channels,
                                   in_size,
                                   ksp_slice)
ksp_slice = ksp_slice * scaling_factor # original fit_untrained() f'n returns this
ksp_slice_tt = to_tensor(ksp_slice)
    
# mask the kspace
ksp_masked_tt = apply_mask(ksp_slice_tt, mask=mask)
    
# convert to torch variable [C, W, H] --> [1, C, W, H]
ksp_masked_tt = np_to_var(ksp_masked_tt.data.cpu().numpy()).type(dtype)

# perform ifft of masked kspace
img_masked_tt = ifft_2d(ksp_masked_tt)

### TODO: start here
out = []
for img in sampled_image2:
    out += [ img[:,:,0].numpy() , img[:,:,1].numpy() ]
lsest = torch.tensor(np.array([out]))

torch.Size([15, 640, 368, 2])


### Perform data consistency step